In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import sklearn
from scipy import stats
from sklearn.metrics import mean_absolute_error 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import median_absolute_error
from datetime import date
from sklearn.preprocessing import MinMaxScaler



In [4]:
df=pd.read_csv('Fractionator.csv')

In [5]:
df['Time']=pd.to_datetime(df['Time'])
df.set_index('Time')

,AI_2020_OVERHEAD_C5'S_MOL,AI_2021_MIDDLE_C7'S_MOL,AI_2022_BOTTOM_C3'S_MOL,FIC_2100_PV_FEED_FURNACE_FUEL_SCFH,FIC-2101_PV_TOP_PRODUCT_MBBL/D,FIC_2102_PV_BOTTOM_PRODUCT_MBBL/D,FI-2005_PV_FEED_FLOW_MBBL/D,FIC-2001_SP_TOP_REFLUX_SETPOINT_MBBL/D,FIC-2001_OP_TOP_REFLUX_OUTPUT_%,FIC_2001_PV_TOP_REFLUX_SETPOINT_MBBL/D,FIC-2002_SP_MIDDLE_PRODUCT_DRAW_SETPOINT_MBBL/D,FIC_2002_OP_MIDDLE_PRODUCT_DRAW_OUTPUT_%,FIC_2002_PVMIDDLE_PRODUCT_DRAW_MBBL/D,FIC_2004_SP_MIDDLE_REFLUX _SETPOINT_MBBL/D,FIC_2004_OP_MIDDLE_REFLUX_OUTPUT_%,FIC_2004_PV_TOP_REFLUX_MBBL/D,QI_2106_PV_MIDDLE_REFLUX_DUTY_BTU/H,TIC_2003_SP_FEED_TEMPERATURE_SETPOINT_DEGF
Time,,,,,,,,,,,,,,,,,,
2009-10-01 08:14:00,2.00000,3.00000,4.00000,8.50000,2.10000,1.70000,5.02153,2.5,41.5425,2.64333,1.5,39.6248,1.55033,8.0,27.0211,8.06381,2.00000,200.0
2009-10-01 08:15:00,2.00360,3.00954,3.99831,8.50735,2.09686,1.71730,4.98768,2.5,40.7619,2.51377,1.5,32.2511,1.43952,8.0,30.9622,8.02407,1.99772,200.0
2009-10-01 08:16:00,2.00870,2.99518,3.97746,8.50977,2.09280,1.72500,4.93990,2.5,39.5543,2.39550,1.5,36.5874,1.55065,8.0,33.4288,7.92434,1.99552,200.0
2009-10-01 08:17:00,2.00385,3.01356,3.97262,8.50838,2.08587,1.71878,4.93333,2.5,39.3031,2.32773,1.5,37.0797,1.50101,8.0,26.2273,8.03058,1.99483,200.0
2009-10-01 08:18:00,1.99852,3.02963,3.98841,8.50365,2.08028,1.71360,4.96880,2.5,40.1886,2.50900,1.5,33.2056,1.48210,8.0,29.6134,8.06181,1.99465,200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-10-05 09:09:00,1.66446,3.09194,4.16930,9.67720,3.29748,2.23989,6.25805,2.5,40.8462,2.58179,1.5,33.5316,1.50803,10.0,68.9767,10.01570,2.81524,201.0
2009-10-05 09:10:00,1.66227,3.09083,4.18922,9.67463,3.31139,2.22584,6.24402,2.5,39.9719,2.42493,1.5,33.9228,1.47920,10.0,55.2307,9.87805,2.81478,201.0
2009-10-05 09:11:00,1.65845,3.07950,4.19189,9.68022,3.29344,2.21896,6.24876,2.5,40.0100,2.47201,1.5,39.5957,1.46495,10.0,56.7539,10.08950,2.81331,201.0


In [6]:
df=df[(np.abs(stats.zscore(df["AI_2020_OVERHEAD_C5'S_MOL"])) < 3)]

In [9]:
df.rename(columns = {"AI_2020_OVERHEAD_C5'S_MOL":"AI_top","AI_2021_MIDDLE_C7'S_MOL":"AI_middle","AI_2022_BOTTOM_C3'S_MOL":"AI_bottom",
                          'FIC_2100_PV_FEED_FURNACE_FUEL_SCFH':'Furnace' ,'FIC_2004_PV_TOP_REFLUX_MBBL/D':'Reflux_top',
                    'FI-2005_PV_FEED_FLOW_MBBL/D':'Feed'
 }, inplace = True)

In [10]:
df_uni=df[['AI_top','Furnace','Feed','Reflux_top',]]

In [11]:
df_cnn=df[['AI_top']]

In [12]:
def df_to_X_y(df, window_size=10):
  df_np = df_cnn.to_numpy()
  X = []
  y = []
  for i in range(len(df_np)-window_size):
    row = [r for r in df_np[i:i+window_size]]
    X.append(row)
    label = df_np[i+window_size][0]
    y.append(label)
  return np.array(X), np.array(y)

In [13]:
X,y= df_to_X_y(df_cnn,window_size=10)
X.shape, y.shape

((5719, 10, 1), (5719,))

In [14]:
split_index = int(len(X) * 0.8)

split_index

4575

In [15]:
X_train = X[:split_index]
X_test = X[split_index:]

y_train = y[:split_index]
y_test = y[split_index:]

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4575, 10, 1), (1144, 10, 1), (4575,), (1144,))

In [12]:

#X_train = X_train.reshape((-1, window_size,window_size,1))
#X_test = X_test.reshape((-1, 2*window_size,1))

y_train = y_train.reshape((-1, 1))
y_test = y_test.reshape((-1, 1))

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4575, 10, 1), (1144, 10, 1), (4575, 1), (1144, 1))

In [13]:
X_train = torch.tensor(X_train).float()
y_train = torch.tensor(y_train).float()
X_test = torch.tensor(X_test).float()
y_test = torch.tensor(y_test).float()
X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([4575, 10, 1]),
 torch.Size([1144, 10, 1]),
 torch.Size([4575, 1]),
 torch.Size([1144, 1]))

In [14]:
from torch.utils.data import Dataset

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, i):
        return self.X[i], self.y[i]
    


In [15]:
train_dataset = TimeSeriesDataset(X_train, y_train)
test_dataset = TimeSeriesDataset(X_test, y_test)

In [16]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'


In [17]:
from torch.utils.data import DataLoader

batch_size = 16

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [18]:
for _, batch in enumerate(train_loader):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)
    print(x_batch.shape, y_batch.shape)
    break

[2023-04-19 22:08:38.950 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:475 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2023-04-19 22:08:39.186 1-8-1-cpu-py36-ml-t3-medium-05a4a7868130c7575335c53b16c7:475 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
torch.Size([16, 10, 1]) torch.Size([16, 1])


In [19]:
x_batch[0]

tensor([[2.0083],
        [2.0058],
        [1.9978],
        [1.9959],
        [2.0056],
        [2.0075],
        [2.0082],
        [2.0056],
        [2.0048],
        [2.0038]])

In [42]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# Define the CNN model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=10, out_channels=16, kernel_size=3)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32 * 22, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 32 * 22)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Prepare the time series data
data = df_cnn.to_numpy()
window_size = 10
X = []
y = []
for i in range(len(data) - window_size):
    X.append(data[i:i+window_size])
    y.append(data[i+window_size])
X = np.array(X).reshape(-1, 1, window_size)
y = np.array(y).reshape(-1, 1)

# Split the data into training and test sets
split_ratio = 0.8
split_index = int(split_ratio * len(X))
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]

# Normalize the data
mean = np.mean(X_train)
std = np.std(X_train)
X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

# Convert the data to PyTorch tensors
X_train = torch.FloatTensor(X_train)
y_train = torch.FloatTensor(y_train)
X_test = torch.FloatTensor(X_test)
y_test = torch.FloatTensor(y_test)

# Train the model
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100
batch_size = 32
train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_train, y_train), batch_size=batch_size)
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if (epoch + 1) % 10 == 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

# Evaluate the model on the test set
predictions = []
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predictions.append(outputs.numpy())
predictions = np.array(predictions).flatten()
predictions = predictions * std + mean
y_test = y_test.numpy() * std + mean
mse = np.mean((predictions - y_test) ** 2)
print("MSE on test set: ", mse)


RuntimeError: Given groups=1, weight of size [16, 10, 3], expected input[32, 1, 10] to have 10 channels, but got 1 channels instead

In [36]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=10, out_channels=64, kernel_size=1)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3)
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(128 ,23, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        x = x.view(-1, 128 * 23)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x




In [37]:
model = CNN()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [38]:
# Train loop
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()


RuntimeError: Given input size: (64x1x1). Calculated output size: (64x1x0). Output size is too small

In [ ]:
# Evaluation
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        # Compute metrics such as MSE or MAE


In [25]:
def train_one_epoch():
    model.train(True)
    print(f'Epoch: {epoch + 1}')
    running_loss = 0.0
    
    for batch_index, batch in enumerate(train_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        output = model(x_batch)
        loss = loss_function(output, y_batch)
        running_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch_index % 100 == 99:  # print every 100 batches
            avg_loss_across_batches = running_loss / 100
            print('Batch {0}, Loss: {1:.3f}'.format(batch_index+1,
                                                    avg_loss_across_batches))
            running_loss = 0.0

In [23]:
def validate_one_epoch():
    model.train(False)
    running_loss = 0.0
    
    for batch_index, batch in enumerate(test_loader):
        x_batch, y_batch = batch[0].to(device), batch[1].to(device)
        
        with torch.no_grad():
            output = model(x_batch)
            loss = loss_function(output, y_batch)
            running_loss += loss.item()

    avg_loss_across_batches = running_loss / len(test_loader)
    
    print('Val Loss: {0:.3f}'.format(avg_loss_across_batches))
    print('***************************************************')
    print()

In [29]:
learning_rate = 0.001
num_epochs = 10
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    train_one_epoch()
    validate_one_epoch()

Epoch: 1


RuntimeError: Calculated padded input size per channel: (1). Kernel size: (3). Kernel size can't be greater than actual input size

In [ ]:
with torch.no_grad():
    predicted_train = model(X_train.to(device)).to('cpu').numpy()

In [ ]:
with torch.no_grad():
    predicted_test = model(X_test.to(device)).to('cpu').numpy()


### new model

In [22]:
   x, y = list(), list()
def split_sequence(sequence, n_steps):
    x, y = list(), list()
    for i in range(len(sequence)):
        
        end_ix = i + n_steps
        
        if end_ix > len(sequence)-1:
            break
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        x.append(seq_x)
        y.append(seq_y)
    return np.array(x), np.array(y)

raw_seq = [10,20,30,40,50,60,70,80,90]
n_steps = 10
train_x,train_y = split_sequence(df_cnn.to_numpy(),n_steps)
#valid_x,valid_y = split_sequence(valid_set.Elec_kW.values,n_steps)

In [23]:
train_x[0]

array([[2.     ],
       [2.0036 ],
       [2.0087 ],
       [2.00385],
       [1.99852],
       [1.9971 ],
       [2.0026 ],
       [2.0144 ],
       [2.01669],
       [2.02368]])

In [ ]:
class ElecDataset(Dataset):
    def __init__(self,feature,target):
        self.feature = feature
        self.target = target
    
    def __len__(self):
        return len(self.feature)
    
    def __getitem__(self,idx):
        item = self.feature[idx]
        label = self.target[idx]
        
        return item,label

In [ ]:
class CNN_ForecastNet(nn.Module):
    def __init__(self):
        super(CNN_ForecastNet,self).__init__()
        self.conv1d = nn.Conv1d(3,64,kernel_size=1)
        self.relu = nn.ReLU(inplace=True)
        self.fc1 = nn.Linear(64*2,50)
        self.fc2 = nn.Linear(50,1)
        
    def forward(self,x):
        x = self.conv1d(x)
        x = self.relu(x)
        x = x.view(-1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        
        return x

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = CNN_ForecastNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)
criterion = nn.MSELoss()

In [ ]:
train = ElecDataset(train_x.reshape(train_x.shape[0],train_x.shape[1],1),train_y)
valid = ElecDataset(valid_x.reshape(valid_x.shape[0],valid_x.shape[1],1),valid_y)
train_loader = torch.utils.data.DataLoader(train,batch_size=2,shuffle=False)
valid_loader = torch.utils.data.DataLoader(train,batch_size=2,shuffle=False)

In [ ]:
train_losses = []
valid_losses = []
def Train():
    
    running_loss = .0
    
    model.train()
    
    for idx, (inputs,labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        preds = model(inputs.float())
        loss = criterion(preds,labels)
        loss.backward()
        optimizer.step()
        running_loss += loss
        
    train_loss = running_loss/len(train_loader)
    train_losses.append(train_loss.detach().numpy())
    
    print(f'train_loss {train_loss}')
    
def Valid():
    running_loss = .0
    
    model.eval()
    
    with torch.no_grad():
        for idx, (inputs, labels) in enumerate(valid_loader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()
            preds = model(inputs.float())
            loss = criterion(preds,labels)
            running_loss += loss
            
        valid_loss = running_loss/len(valid_loader)
        valid_losses.append(valid_loss.detach().numpy())
        print(f'valid_loss {valid_loss}')

In [ ]:
epochs = 200
for epoch in range(epochs):
    print('epochs {}/{}'.format(epoch+1,epochs))
    Train()
    Valid()
    gc.collect()